In [1]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np

df = pd.read_csv('df_ligas_tratado.csv')

In [11]:
# Pré-processamento: Empilhar os dados (Criar 2 linhas por jogo)

# Perspectiva do Time da Casa
df_casa = df.copy()
df_casa["Time"] = df_casa["teams.home.name"]
df_casa["Oponente"] = df_casa["teams.away.name"]
df_casa["Gols_Feitos"] = df_casa["goals.home"]
df_casa["Gols_Sofridos"] = df_casa["goals.away"]
df_casa["Local"] = 1 # Jogando em Casa
df_casa["Vitoria"] = (df_casa["winner.team"] == df_casa["Time"]).astype(int)
df_casa = df_casa[["league.name", "Time", "Oponente", "Local", "Vitoria", "Gols_Feitos", "Gols_Sofridos"]]

# Perspectiva do Time Visitante
df_visitante = df.copy()
df_visitante["Time"] = df_visitante["teams.away.name"]
df_visitante["Oponente"] = df_visitante["teams.home.name"]
df_visitante["Gols_Feitos"] = df_visitante["goals.away"]
df_visitante["Gols_Sofridos"] = df_visitante["goals.home"]
df_visitante["Local"] = 0 # Jogando fora de casa
df_visitante["Vitoria"] = (df_visitante["winner.team"] == df_visitante["Time"]).astype(int)
df_visitante = df_visitante[["league.name", "Time", "Oponente", "Local", "Vitoria", "Gols_Feitos", "Gols_Sofridos"]]

# Juntando resultados
df_completo = pd.concat([df_casa, df_visitante], ignore_index=True)


In [20]:
# Modelagem

# Modelo A: GLM Geral (Logística) - "Visão Agregada"
# Pergunta: Em média, jogar em casa aumenta a chance de vitória?
modelo_glm = smf.logit("Vitoria ~ Local", data=df_completo).fit(disp=0)

# Modelo B: Simulando a visão "Por Liga"
# Vamos rodar o mesmo modelo separadamente para cada liga para ver a variação dos coeficientes
resultados_por_liga = []

ligas = df_completo["league.name"].unique()

for liga in ligas:
    df_liga = df_completo[df_completo["league.name"] == liga]
    modelo_liga = smf.logit("Vitoria ~ Local", data=df_liga).fit(disp=0)

    coef_local = modelo_liga.params["Local"]
    odds_ratio = np.exp(coef_local)
    p_valor = modelo_liga.pvalues["Local"]

    resultados_por_liga.append({
        'Liga': liga,
        'Coeficiente_Local': coef_local,
        'Odds_Ratio_Casa': odds_ratio,
        'P_Valor': p_valor
    })

df_resultados = pd.DataFrame(resultados_por_liga)

# Adicionar resultados do Modelo A
coef_geral = modelo_glm.params['Local']
or_geral = np.exp(coef_geral)
p_geral = modelo_glm.pvalues["Local"]
df_resultados.loc[len(df_resultados)] = ['MÉDIA GERAL (GLM)', coef_geral, or_geral, p_geral]

# Resultados
print("--- Comparação do Efeito 'Jogar em Casa' na Vitória ---")
print(df_resultados.sort_values(by="Odds_Ratio_Casa", ascending=False).to_markdown(index=False, floatfmt=".3f"))
print("\nNota: Odds Ratio > 1 indica que jogar em casa aumenta a chance de vitória.")
print(f"Exemplo Geral: Odds Ratio de {or_geral:.2f} significa que a chance de vitória em casa é {or_geral:.2f} vezes a chance de vitória fora.")

--- Comparação do Efeito 'Jogar em Casa' na Vitória ---
| Liga              |   Coeficiente_Local |   Odds_Ratio_Casa |   P_Valor |
|:------------------|--------------------:|------------------:|----------:|
| Serie A           |               0.850 |             2.339 |     0.000 |
| La Liga           |               0.706 |             2.027 |     0.000 |
| MÉDIA GERAL (GLM) |               0.600 |             1.823 |     0.000 |
| Premier League    |               0.579 |             1.784 |     0.000 |
| Bundesliga        |               0.577 |             1.780 |     0.001 |
| Ligue 1           |               0.224 |             1.251 |     0.181 |

Nota: Odds Ratio > 1 indica que jogar em casa aumenta a chance de vitória.
Exemplo Geral: Odds Ratio de 1.82 significa que a chance de vitória em casa é 1.82 vezes a chance de vitória fora.
